In [339]:
import numpy as np
import pandas as pd
import random
from random import randint
from random import uniform
from sklearn.model_selection import train_test_split
import pickle


numeric_size = 3
num_of_patients = 20000

def get_rand_visit_codes():
    #num_of_codes_in_visit = randint(1,5)
    #visit_c = random.sample(range(0,68), num_of_codes_in_visit)
    num_of_codes_in_visit = 2
    visit_c = []
    #print (visit_c)
    return visit_c
ite_rand = 0
def get_rand_visit_numerics():
    #[Diastolic,Systolic, pulse]
    global ite_rand
    if ite_rand == 0:
        visit_n = [uniform(60,90),uniform(90,160),uniform(60,100)]
    else:
        visit_n = [uniform(90,110),uniform(160,180),uniform(100,120)]
    #print (visit_n)
    return visit_n

def create_patient():
    num_of_visits = randint(5,20)
    global ite_rand
    if ite_rand == 0:
        ite_rand = 1
    else:
        ite_rand = 0
    anomaly_patient_count = 0
    patient_c = []
    for k in range(0,num_of_visits,1):
        patient_c.append(get_rand_visit_codes())
    #print (patient_c)

    patient_n = []
    for k in range(0,num_of_visits,1):
        visit_numerics = get_rand_visit_numerics()
        patient_n.append(visit_numerics)
        anomaly_patient_count = anomaly_patient_count + validate_numerics_for_anomaly(visit_numerics)
        
    #print ("New patient "+str(len(target))+", # of visits: "+str(num_of_visits))
    #print ("Number of anomalies: "+ str(anomaly_patient_count))
    if anomaly_patient_count/num_of_visits > 0.7:
        target.append(1)
        #print ("Target: "+ str(1))
    else:
        target.append(0)
        #print ("Target: "+ str(0))
    #print("Target value in list: "+str(target[len(target)-1]))
    return [patient_c,patient_n,None,anomaly_patient_count]

def validate_numerics_for_anomaly(numerics):
    #[Diastolic,Systolic, pulse]
    if (numerics[0] >= 100 or numerics[0] <= 55):
        return 1
    elif (numerics[1] >= 170 or numerics[1] <= 85):
        return 1
    elif (numerics[2] >= 110 or numerics[2] <= 55):
        return 1
    else:
        return 0


In [340]:
target=[]
df = pd.DataFrame([create_patient()], columns=['codes','numerics','to_event','anomalies'])

In [341]:
k=1
for k in range(1,num_of_patients,1):
    if (k == 1000 or k == 3000 or k == 5000 or k == 8000 or k == 10000 or k == 15000):
        print ("created_patient: "+str(k))
    df.loc[k] = create_patient()
#print(target)
print(str(target.count(1)))
print(str(target.count(0)))

SyntaxError: invalid syntax (<ipython-input-341-ff9280892136>, line 3)

In [321]:
sort_indicies = np.argsort(list(map(len, df['codes'].tolist())))
sort_indicies

array([   0, 5196, 5175, ..., 5146, 4140, 6196])

In [322]:
all_data =df.iloc[sort_indicies].reset_index()

In [325]:
tg = pd.DataFrame([target[0]], columns=['target'])
k=1
for k in range(1,num_of_patients,1):
    tg.loc[k] = target[k]

In [326]:
all_targets = tg.iloc[sort_indicies].reset_index()

In [328]:
train_proportion=0.7
data_train,data_test = train_test_split(all_data, train_size=train_proportion, random_state=12345)
target_train,target_test = train_test_split(all_targets, train_size=train_proportion, random_state=12345)


In [329]:
out_directory= 'data'

In [330]:
data_train.sort_index().to_pickle(out_directory+'/data_train.pkl')
data_test.sort_index().to_pickle(out_directory+'/data_test.pkl')

In [331]:
target_train.sort_index().to_pickle(out_directory+'/target_train.pkl')
target_test.sort_index().to_pickle(out_directory+'/target_test.pkl')

In [332]:
from icd9 import ICD9
tree = ICD9('codes_pretty_printed.json')

In [333]:
labels_icd9 = []
ids = []


for k in range(390,459,1):
    ids.append(k-390)
    if tree.find(str(k)) is not None:
        #print(tree.find(str(k)).description)
        labels_icd9.append(tree.find(str(k)).description)
    else:
        #print(tree.find('390-459').description+str(k))
        labels_icd9.append(tree.find('390-459').description+str(k))
        
#print(labels_icd9)        
#print(ids)        
ids.append(70)
ids.append(71)
ids.append(72)

labels_icd9.append('Diastolic')
labels_icd9.append('Systolic')
labels_icd9.append('Pulse')

types = dict(zip(ids,labels_icd9))

In [334]:
#print(types)

In [335]:
pickle.dump(types, open(out_directory+'/dictionary.pkl', 'wb'), -1)

In [336]:
data_test.sort_index()

,index,codes,numerics,to_event,anomalies
2,5175,"[[], [], [], [], []]","[[67.68150161083248, 153.76665801499163, 79.36...",None,0
15,5005,"[[], [], [], [], []]","[[81.06039774913667, 153.1621206784753, 85.206...",None,0
20,4923,"[[], [], [], [], []]","[[88.29233488513962, 102.34908448810896, 91.87...",None,0
22,5208,"[[], [], [], [], []]","[[100.57784282306491, 162.0814965244883, 114.9...",None,5
23,5214,"[[], [], [], [], []]","[[96.53830774962401, 173.5223603722884, 110.32...",None,4
26,5512,"[[], [], [], [], []]","[[93.20427997883746, 162.17938493292976, 103.5...",None,3
28,5487,"[[], [], [], [], []]","[[79.42798017021008, 159.12580074117392, 87.12...",None,0
29,5473,"[[], [], [], [], []]","[[76.79080416531902, 118.75509480393833, 62.63...",None,0
30,5462,"[[], [], [], [], []]","[[108.28277836118552, 172.69301176850192, 102....",None,3
32,5418,"[[], [], [], [], []]","[[94.76626360553328, 170.6579713497439, 118.43...",None,5


In [337]:
target_test.sort_index()

,index,target
2,5175,0
15,5005,0
20,4923,0
22,5208,1
23,5214,1
26,5512,0
28,5487,0
29,5473,0
30,5462,0
32,5418,1


In [338]:
#data_test.sort_index().loc[13]